In [2]:
class LSTMNet(nn.Module):
    def __init__(self,vocab_size,embedding_dim,hidden_dim,layer_dim,output_dim):
        super(LSTMNet,self).__init__()
        self.hidden_dim = hidden_dim #LSTM神经元个数
        self.layer_dim = layer_dim  #LSTM层数
        #对文本进行词向量处理  
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        #LSTM+全连接层 
        self.lstm = nn.LSTM(embedding_dim,hidden_dim,layer_dim,batch_first=True) 
        self.fc1 = nn.Linear(hidden_dim,output_dim) 
    def forward(self,x):   
        embeds = self.embedding(x) 
        r_out,(h_n,h_c) = self.lstm(embeds, None)
        out = self.fc1(r_out[:,-1,:])
        return out  

In [3]:
import pandas as pd 
import jieba
import re 
import string 
import torch.optim as optim
import pandas as pd 
import jieba
import re 
import jieba
import string 
from sklearn.metrics import precision_recall_fscore_support
import torch 
from torch import nn 
import torch.nn.functional as F 
import torch.optim as optim
import torch.utils.data as Data
from torchtext import data
from torchtext.vocab import Vectors
from torchtext.legacy.data import Field,TabularDataset,Iterator,BucketIterator 
lstmmodel = torch.load(r"C:\Users\xumin\深度学习作业/lstmmodel.pkl") 
train_dataset=pd.read_csv(r'C:\Users\xumin\Desktop\Python\challenging\data\train.csv')
test_dataset=pd.read_csv(r'C:\Users\xumin\Desktop\Python\challenging\data\test.csv')  

In [4]:
mytokenize = lambda x: x.split()
TEXT = Field(sequential=True,tokenize=mytokenize,include_lengths=True,use_vocab=True,batch_first=True)
LABEL=Field(sequential=False,use_vocab=False,pad_token=None,unk_token=None) 
text_data_fields = [("label",LABEL),("title",TEXT)]   
traindata,validdata,testdata=TabularDataset.splits(path=r"C:\Users\xumin\大作业",format="csv",train="train2.csv",fields=text_data_fields,
                                                  validation="val2.csv",test="test2.csv",skip_header=True)     

In [6]:
BATCH_SIZE = 64 
train_iter = BucketIterator(traindata,batch_size = BATCH_SIZE)
val_iter = BucketIterator(validdata,batch_size = BATCH_SIZE) 
test_iter = BucketIterator(testdata,batch_size = BATCH_SIZE)    

In [8]:
TEXT.build_vocab(traindata,max_size=20000,vectors=None)
LABEL.build_vocab(traindata) 
vocab_size = len(TEXT.vocab)
embedding_dim = 100
hidden_dim = 100      
layer_dim = 1   
output_dim = 2  
lstmmodel = LSTMNet(vocab_size,embedding_dim,hidden_dim,layer_dim,output_dim)
lstmmodel  

LSTMNet(
  (embedding): Embedding(15473, 100)
  (lstm): LSTM(100, 100, batch_first=True)
  (fc1): Linear(in_features=100, out_features=2, bias=True)
)

In [20]:
for step,batch in enumerate(test_iter):
    textdata,target = batch.title[0],batch.label.view(-1)    
    out = lstmmodel(textdata)
    pre_lab = torch.argmax(out,1)   
    test_y_all=torch.cat((test_y_all,target))  
    pre_lab_all=torch.cat((pre_lab_all,pre_lab))   
    print("真正标签：",target)
    print("猜测：",pre_lab)  
    break   

真正标签： tensor([0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])
猜测： tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [23]:
for step,batch in enumerate(test_iter):
    textdata,target = batch.title[0],batch.label.view(-1)    
    out = lstmmodel(textdata)
    pre_lab = torch.argmax(out,1)   
    test_y_all=torch.cat((test_y_all,target))  
    pre_lab_all=torch.cat((pre_lab_all,pre_lab))   
    print("真正标签：",target)
    print("猜测：",pre_lab)  
    break   

真正标签： tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
猜测： tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
